In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import glob
import os
import re

from tqdm.notebook import tqdm
tqdm.pandas()

from IPython.core.display import Markdown,display, HTML, Latex
import qgrid

import seaborn as sns
import matplotlib.patheffects as pe
%matplotlib inline

/home/qudratealahyratu/anaconda3/envs/scifact/lib/python3.7/site-packages/tqdm/std.py:670: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [3]:
import spacy

# nlp = spacy.load("en_core_web_trf")

from spacy import displacy
from nltk import word_tokenize
import nltk

In [4]:
from pattern.en import quantify, numerals, number,reflect

from nltk.corpus import wordnet as wn_nltk
from itertools import chain

from pattern.en import NOUN, VERB, ADJECTIVE, ADVERB, NOUN, VERB, ADJ, ADV, PRON, DET, PREP, ADP, NUM, CONJ, INTJ, PRT, PUNC, X
from pattern.en import  POS, CHUNK, INFINITIVE, PRESENT, PAST, FUTURE, FIRST, SECOND, THIRD,SINGULAR, PLURAL, SG, PL, PROGRESSIVE, PARTICIPLE, WORD
from pattern.text import UNIVERSAL
from pattern.en import conjugate, lemma, lexeme,PRESENT,SG, verbs, PARTICIPLE, ngrams
from pattern.en import wordnet as wn_pattern

from pattern.en import tree
from pattern.text.tree import Word
from pattern.en import parse, parsetree, tag, pprint

In [5]:
nlp_sci_brt = spacy.load("en_core_sci_scibert")

## Load original claims by tech terms

In [6]:
loc_df_scispacy_sentence_word_unq_ner_abr_filtered ='../../dfs_generated/linguistic/df_scispacy_sentence_word_unq_ner_abr_filtered.pkl'

In [7]:
df_sentence_word_unq_ner_abr_filtered = pd.read_pickle(loc_df_scispacy_sentence_word_unq_ner_abr_filtered)

In [8]:
df_sentence_word_unq_ner_abr_filtered

,ner_text,ner_label,ner_model,claim,start_char,end_char,org_label,list_rationales,data_source
0,UK,GENE_OR_GENE_PRODUCT,en_ner_bionlp13cg_md,1 in 5 million in UK have abnormal PrP positiv...,18,20,REFUTES,"[RESULTS Of the 32,441 appendix samples 16 wer...",train
1,PrP,GENE_OR_GENE_PRODUCT,en_ner_bionlp13cg_md,1 in 5 million in UK have abnormal PrP positiv...,35,38,REFUTES,"[RESULTS Of the 32,441 appendix samples 16 wer...",train
2,genomes,SO,en_ner_craft_md,"1,000 genomes project enables mapping of genet...",6,13,SUPPORTS,"[In conclusion, uncommon or rare genetic varia...",dev
3,genetic sequence,SO,en_ner_craft_md,"1,000 genomes project enables mapping of genet...",41,57,SUPPORTS,"[In conclusion, uncommon or rare genetic varia...",dev
4,variants,SO,en_ner_craft_md,"1,000 genomes project enables mapping of genet...",139,147,SUPPORTS,"[In conclusion, uncommon or rare genetic varia...",dev
...,...,...,...,...,...,...,...,...,...
1724,A20,PROTEIN,en_ner_jnlpba_md,siRNA knockdown of A20 slows tumor progression...,19,22,SUPPORTS,[The tumorigenic potential of GSCs was decreas...,train
1725,tumor,CANCER,en_ner_bionlp13cg_md,siRNA knockdown of A20 slows tumor progression...,29,34,SUPPORTS,[The tumorigenic potential of GSCs was decreas...,train
1726,murine,TAXON,en_ner_craft_md,siRNA knockdown of A20 slows tumor progression...,61,77,SUPPORTS,[The tumorigenic potential of GSCs was decreas...,train
1727,β-sheet,PROTEIN,en_ner_jnlpba_md,β-sheet opening occurs during pleurotolysin po...,0,7,SUPPORTS,[The major conformational changes in PlyB are ...,train


In [9]:
df_sentence_word_unq_ner_abr_filtered['term_in_claim'] = df_sentence_word_unq_ner_abr_filtered.apply(lambda x: x['claim'][x['start_char']:x['end_char']], axis=1)

In [10]:
df_sentence_word_unq_ner_abr_filtered[ df_sentence_word_unq_ner_abr_filtered['claim'] == 'A T helper 2 cell (Th2) environment impedes disease development in patients with systemic lupus erythematosus (SLE).']

,ner_text,ner_label,ner_model,claim,start_char,end_char,org_label,list_rationales,data_source,term_in_claim
17,T helper 2 cell,GGP,en_ner_craft_md,A T helper 2 cell (Th2) environment impedes di...,2,23,REFUTES,"[Thus, in Lyn(-/-) mice, basophils and IgE aut...",train,T helper 2 cell (Th2)
18,patients,ORGANISM,en_ner_bionlp13cg_md,A T helper 2 cell (Th2) environment impedes di...,67,75,REFUTES,"[Thus, in Lyn(-/-) mice, basophils and IgE aut...",train,patients
19,systemic lupus erythematosus,DISEASE,en_ner_bc5cdr_md,A T helper 2 cell (Th2) environment impedes di...,81,115,REFUTES,"[Thus, in Lyn(-/-) mice, basophils and IgE aut...",train,systemic lupus erythematosus (SLE)


In [11]:
'A T helper 2 cell (Th2) environment impedes disease development in patients with systemic lupus erythematosus (SLE).'[81:115]

'systemic lupus erythematosus (SLE)'

In [12]:
df_grp_sentence_word_unq_ner_abr_filtered = df_sentence_word_unq_ner_abr_filtered.loc[:, :].groupby('claim')

In [13]:
dic_map_strpos_patternpos = {'NOUN' :NOUN, 'VERB' : VERB, 'ADJECTIVE': ADJECTIVE, 'ADVERB' : ADVERB}

In [14]:
css_style = """
        <style>
        .org_claim_container {
          background-color: #f0f6fc;
          color: #000000cc;
          border: 2px ridge #72aee655;
          border-radius: 10px;
          margin: 5px;
          padding: 10px;
        }

        .term_container{
          background-color: #faebf6;
          color: #af0000dc;
          border-radius: 5px;
          border: 2px ridge #faebf613;
          font-size: 12pt;
          font-weight: bold;
        }

        .non_term_container{
          background-color: #fff7fd;
          color: #001630;
          font-size: 12pt;
        }   
        
        .meta_info_token{
        display: block;
        background-color: #00a6f309;
        border: 1px ridge #00a6f333;
        margine : 5px;
        padding : 5px;
        font-weight: 400;
        text-shadow: 0.2px 0.2px #00000055;
        box-shadow: 1px 1px #00660022;
        }
        </style>
    """

gen_html = css_style

list_dic_of_sent_tag_info = []

for cur_claim, cur_df_group in tqdm(df_grp_sentence_word_unq_ner_abr_filtered):
    doc_sci_brt = nlp_sci_brt(cur_claim)
    list_term_ranges = cur_df_group[['start_char', 'end_char']].values
    gen_html += '<div class="org_claim_container">'
    
    
    
    previous_term_end = 0
    for cur_indx, cur_term_range in enumerate(list_term_ranges):
        
        gen_html += '<span class = "non_term_container">'+ cur_claim[previous_term_end:cur_term_range[0]]+'</span>'
        gen_html += '<span class = "term_container">'+ cur_claim[cur_term_range[0]:cur_term_range[1]]+'</span>'
        
        doc_sci_brt = nlp_sci_brt(cur_claim[previous_term_end:cur_term_range[0]])
        
        
        for cur_token in doc_sci_brt:
            dic_token_all_info = {}
#             gen_html += '<span class = "meta_info_token">'+ str({'<b>token_text': cur_token.text, '</b>token_lemma' : cur_token.lemma_, 
#                                                                  'pos' : cur_token.pos_,'dep' : cur_token.dep_, 
#                                                                  'tag' : cur_token.tag_,
#                                                                  'tag_explain' : spacy.explain(cur_token.tag_),
#                                                                 })+'</span>'
            
            
            dic_token_all_info['claim'] = cur_claim, 
            dic_token_all_info['token'] = cur_token.text 
            
#             print('org label ', set(cur_df_group['org_label']))
            dic_token_all_info['org_label'] = list(set(cur_df_group['org_label']))[0]
            dic_token_all_info['list_rationales'] = list(cur_df_group['list_rationales'])[0]
            dic_token_all_info['data_source'] = list(set(cur_df_group['data_source']))[0]
#             dic_token_all_info['claim'] = 
            
#             print('\n\n#### spacy ####')
            
#             print({'token_text': cur_token.text, 'token_lemma' : cur_token.lemma_, 
#                                                                  'pos' : cur_token.pos_,'dep' : cur_token.dep_, 
#                                                                  'tag' : cur_token.tag_,
#                                                                  'tag_explain' : spacy.explain(cur_token.tag_),                                                                 
#                                                                 })
            
            
            dic_token_all_info['token_text_spacy'] = cur_token.text 
            dic_token_all_info['token_lemma_spacy'] = cur_token.lemma_ 
            dic_token_all_info['pos_spacy'] = cur_token.pos_
            dic_token_all_info['dep_spacy'] = cur_token.dep_
            dic_token_all_info['tag_spacy'] = cur_token.tag_
            dic_token_all_info['tag_explain_spacy'] = spacy.explain(cur_token.tag_),                                                                 
                                                                
#             print('\n#### NLTK ####')
            syns_wn_nltk = wn_nltk.synsets(cur_token.text)
            synset_wn_nltk = list(set(chain.from_iterable([word.lemma_names() for word in syns_wn_nltk])))
#             print('nltk syn : ', cur_token.text, ' >>> ', synset_wn_nltk)
            dic_token_all_info['wn_nltk_syn'] = synset_wn_nltk
            
            
#             print('morph :', cur_token.morph.to_dict())
            dic_token_all_info['morf_nltk'] = cur_token.morph.to_dict()
            
#             print('\n#### Pattern ####')
            pattern_syns_list = []
            
            try:
                pattern_syns_list.append(numerals(cur_token.text))
            except :
                print()
#             word_pattern = Word(cur_claim, cur_token.text)
#             print('word_pattern > ', word_pattern.string)


#             print('lemma', ' > ', lemma(cur_token.text))
            
            
            if(cur_token.pos_ in ['NOUN', 'VERB', 'ADJECTIVE', 'ADVERB'] ):
                cur_pos = str(cur_token.pos_).upper()
#                 print("cur_token > ", cur_token.text)
                
                for synset in wn_pattern.synsets(cur_token.text, pos=dic_map_strpos_patternpos[cur_token.pos_]):
#                     print("Token pos : ", synset.pos)
#                     print("Token similar : ", synset.similar())
#                     print("synonyms : %s" % synset.synonyms)      
#                     print("pos  : %s" % synset.lexname )    
                    pattern_syns_list += synset.synonyms
            pattern_syns_list = list(set(pattern_syns_list))
            dic_token_all_info['wn_pattern_syn'] = pattern_syns_list
#             print('pattern synonyms :', pattern_syns_list)
            
            list_dic_of_sent_tag_info.append(dic_token_all_info)
        previous_term_end = cur_term_range[1]
        
    gen_html += '<span class = "non_term_container">'+cur_claim[previous_term_end:]+'</span>'
    gen_html += '</div>'
    

    


In [1]:
HTML(gen_html)

NameError: name 'HTML' is not defined

In [16]:
df_sentence_word_unq_ner_abr_filtered['claim'].value_counts()

The YAP1 and TEAD complex tanslocates into the nucleus where it interacts with transcription factors and DNA-binding proteins that modulate target gene transcription.             7
Replacement of histone H2A with H2A.Z slows gene activation in yeasts by stabilizing +1 nucleosomes.                                                                               7
Ectopic expression of Sall4, Nanog, Esrrb, and Lin28 generates induced pluripotent stem cells from mouse embryonic fibroblasts more efficiently than other factor combinations.    6
Depletion of T-helper 17 (Th17) cells during simian immunodeficiency virus (SIV) infection increases dissemination of Salmonella Typhimurium from the gut.                         6
Depletion of T-helper 17 (Th17) cells during simian immunodeficiency virus (SIV) infection decreases dissemination of Salmonella Typhimurium from the gut.                         6
                                                                                               

## Load all paraphrased claim previous all (paws, pegasus...) model

In [17]:
loc_df_all_model_all_claim_1st_itter = '../../dfs_generated/paraphrased/t5_no_fine_tune_generated_claim_all_model_df_full_1.pkl'

In [18]:
df_all_model_all_claim_1st_itter = pd.read_pickle(loc_df_all_model_all_claim_1st_itter)

In [19]:
df_all_model_all_claim_1st_itter.columns

Index(['org_claim', 'ground_label', 'ground_list_rationales', 'source',
       'org_count_support', 'org_count_refute', 'org_list_supported_ids',
       'org_list_refuted_ids', 'org_list_supported_confidence',
       'org_list_refuted_confidence', 'org_list_supported_confidence_mean',
       'org_list_refuted_confidence_mean', 'org_comment', 'gen_claim',
       'gen_count_support', 'gen_count_refute', 'gen_list_supported_ids',
       'gen_list_refuted_ids', 'gen_list_supported_confidence',
       'gen_list_refuted_confidence', 'gen_list_supported_confidence_mean',
       'gen_list_refuted_confidence_mean', 'gen_comment', 'common_all',
       'common_support_refute', 'common_refute_support',
       'common_support_support', 'common_refute_refute', 'model',
       'parrot_score'],
      dtype='object')

In [20]:
df_all_model_all_claim_1st_itter['model'].unique()

array(['paws_base_no_fine_tune', 'parasci_base_no_fine_tune',
       'tapaco_base_no_fine_tune', 'parrot_base_no_fine_tune',
       'parrot_div_base_no_fine_tune', 'pegasus_base_no_fine_tune'],
      dtype=object)

In [21]:
df_all_model_all_claim_1st_itter[['org_claim', 'gen_claim', 'org_comment', 'gen_comment', 'model']]

,org_claim,gen_claim,org_comment,gen_comment,model
0,1 in 5 million in UK have abnormal PrP positiv...,1 out of 5 million people in the UK has an abn...,no result,no result,paws_base_no_fine_tune
1,1 in 5 million in UK have abnormal PrP positiv...,1 in 5 millions in the UK have an abnormal PrP...,no result,no result,paws_base_no_fine_tune
2,1 in 5 million in UK have abnormal PrP positiv...,In the UK 1 in 5 million have abnormal PrP pos...,no result,no result,paws_base_no_fine_tune
3,1 in 5 million in UK have abnormal PrP positiv...,1 in 5 million in the UK have abnormal PrP pos...,no result,no result,paws_base_no_fine_tune
4,1 in 5 million in UK have abnormal PrP positiv...,1 in 5 million people in the UK have abnormal ...,no result,no result,paws_base_no_fine_tune
...,...,...,...,...,...
19440,mTORC2 regulates intracellular cysteine levels...,The mTORC2 regulates cysteine levels.,no result,no result,pegasus_base_no_fine_tune
19441,mTORC2 regulates intracellular cysteine levels...,The mTORC2 regulates the levels of cysteine.,no result,no result,pegasus_base_no_fine_tune
19442,mTORC2 regulates intracellular cysteine levels...,mTORC2 regulates cysteine levels.,no result,no result,pegasus_base_no_fine_tune
19443,mTORC2 regulates intracellular cysteine levels...,mTORC2 regulates the levels of cysteine.,no result,no result,pegasus_base_no_fine_tune


In [22]:
#df_selected_model_all_claim_1st_itter = df_all_model_all_claim_1st_itter[df_all_model_all_claim_1st_itter['model'] == 'paws_base_no_fine_tune']
df_selected_model_all_claim_1st_itter = df_all_model_all_claim_1st_itter.copy()

In [23]:
df_selected_model_all_claim_1st_itter

,org_claim,ground_label,ground_list_rationales,source,org_count_support,org_count_refute,org_list_supported_ids,org_list_refuted_ids,org_list_supported_confidence,org_list_refuted_confidence,...,gen_list_supported_confidence_mean,gen_list_refuted_confidence_mean,gen_comment,common_all,common_support_refute,common_refute_support,common_support_support,common_refute_refute,model,parrot_score
0,1 in 5 million in UK have abnormal PrP positiv...,REFUTES,"[RESULTS Of the 32,441 appendix samples 16 wer...",train,0,0,[],[],[],[],...,0.0,0.0,no result,0,0,0,0,0,paws_base_no_fine_tune,NaN
1,1 in 5 million in UK have abnormal PrP positiv...,REFUTES,"[RESULTS Of the 32,441 appendix samples 16 wer...",train,0,0,[],[],[],[],...,0.0,0.0,no result,0,0,0,0,0,paws_base_no_fine_tune,NaN
2,1 in 5 million in UK have abnormal PrP positiv...,REFUTES,"[RESULTS Of the 32,441 appendix samples 16 wer...",train,0,0,[],[],[],[],...,0.0,0.0,no result,0,0,0,0,0,paws_base_no_fine_tune,NaN
3,1 in 5 million in UK have abnormal PrP positiv...,REFUTES,"[RESULTS Of the 32,441 appendix samples 16 wer...",train,0,0,[],[],[],[],...,0.0,0.0,no result,0,0,0,0,0,paws_base_no_fine_tune,NaN
4,1 in 5 million in UK have abnormal PrP positiv...,REFUTES,"[RESULTS Of the 32,441 appendix samples 16 wer...",train,0,0,[],[],[],[],...,0.0,0.0,no result,0,0,0,0,0,paws_base_no_fine_tune,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19440,mTORC2 regulates intracellular cysteine levels...,SUPPORTS,[mTORC2 phosphorylates serine 26 at the cytoso...,dev,0,0,[],[],[],[],...,0.0,0.0,no result,0,0,0,0,0,pegasus_base_no_fine_tune,NaN
19441,mTORC2 regulates intracellular cysteine levels...,SUPPORTS,[mTORC2 phosphorylates serine 26 at the cytoso...,dev,0,0,[],[],[],[],...,0.0,0.0,no result,0,0,0,0,0,pegasus_base_no_fine_tune,NaN
19442,mTORC2 regulates intracellular cysteine levels...,SUPPORTS,[mTORC2 phosphorylates serine 26 at the cytoso...,dev,0,0,[],[],[],[],...,0.0,0.0,no result,0,0,0,0,0,pegasus_base_no_fine_tune,NaN
19443,mTORC2 regulates intracellular cysteine levels...,SUPPORTS,[mTORC2 phosphorylates serine 26 at the cytoso...,dev,0,0,[],[],[],[],...,0.0,0.0,no result,0,0,0,0,0,pegasus_base_no_fine_tune,NaN


## Rules

In [26]:
def check_terms_wrt_org_claim (cur_org_claim_df_grp_with_terms, cur_gen_claim):
    #print(cur_org_claim_df_group['claim'])
    dic_container_result = {
        'org_claim' : cur_org_claim_df_grp_with_terms['claim'][0], 
        'gen_claim' : cur_gen_claim,
        'count_exists_term' : 0,
        'count_exists_term_ignore_case' : 0,
        'count_ner_term' : 0,
        'count_ner_term_ignore_case' : 0,
        'count_for_org_claim' : len(cur_org_claim_df_grp_with_terms),
        }
    if len(cur_org_claim_df_grp_with_terms.columns) < 2:
        dic_container_result['count_for_org_claim'] = 0
        return dic_container_result
    for cur_term_row in cur_org_claim_df_grp_with_terms.itertuples():
        if cur_term_row.term_in_claim in cur_gen_claim:
            dic_container_result['count_exists_term'] += 1
            
        if cur_term_row.term_in_claim.lower() in cur_gen_claim.lower():
            dic_container_result['count_exists_term_ignore_case'] += 1
            
        if cur_term_row.ner_text in cur_gen_claim:
            dic_container_result['count_ner_term'] += 1
            
        if cur_term_row.ner_text.lower() in cur_gen_claim.lower():
            dic_container_result['count_ner_term_ignore_case'] += 1
            
    return dic_container_result


In [27]:
list_count_only_term = []
for cur_row_gen_claim in tqdm(df_selected_model_all_claim_1st_itter[['org_claim', 'gen_claim']].to_numpy()):
    cur_org_claim_of_gen_claim = cur_row_gen_claim[0]
    cur_gen_claim = cur_row_gen_claim[1]
    try:
        cur_org_claim_df_group = df_grp_sentence_word_unq_ner_abr_filtered.get_group(cur_org_claim_of_gen_claim)
    except KeyError:
        claim_with_no_term = [{'claim' : cur_org_claim_of_gen_claim}]
        cur_org_claim_df_group = pd.DataFrame(claim_with_no_term)    
    cur_org_claim_df_group.reset_index(drop=True, inplace=True)
    cur_res_dic_count = check_terms_wrt_org_claim(cur_org_claim_df_group, cur_gen_claim)
    list_count_only_term.append(cur_res_dic_count)

In [28]:
df_all_claim_selected_model_term_stat = pd.DataFrame(list_count_only_term)

In [29]:
df_all_claim_selected_model_term_stat.to_pickle('../../dfs_generated/linguistic/df_all_claim_selected_model_term_stat.pkl.')

In [30]:
df_all_claim_selected_model_term_stat

,org_claim,gen_claim,count_exists_term,count_exists_term_ignore_case,count_ner_term,count_ner_term_ignore_case,count_for_org_claim
0,1 in 5 million in UK have abnormal PrP positiv...,1 out of 5 million people in the UK has an abn...,2,2,2,2,2
1,1 in 5 million in UK have abnormal PrP positiv...,1 in 5 millions in the UK have an abnormal PrP...,2,2,2,2,2
2,1 in 5 million in UK have abnormal PrP positiv...,In the UK 1 in 5 million have abnormal PrP pos...,2,2,2,2,2
3,1 in 5 million in UK have abnormal PrP positiv...,1 in 5 million in the UK have abnormal PrP pos...,2,2,2,2,2
4,1 in 5 million in UK have abnormal PrP positiv...,1 in 5 million people in the UK have abnormal ...,2,2,2,2,2
...,...,...,...,...,...,...,...
3468,cSMAC formation enhances weak ligand signalling.,cSMAC formation strengthens weak ligand signaling,1,1,1,1,1
3469,cSMAC formation enhances weak ligand signalling.,The formation of cSMAC enhances weak linkage s...,1,1,1,1,1
3470,mTORC2 regulates intracellular cysteine levels...,mTORC2 regulates intracellular cysteine levels...,3,3,3,3,3
3471,mTORC2 regulates intracellular cysteine levels...,mTORC2 regulates intracellular cysteine levels...,3,3,3,3,3


In [31]:
df_all_claim_selected_model_term_stat[df_all_claim_selected_model_term_stat['count_for_org_claim'] == 0]

,org_claim,gen_claim,count_exists_term,count_exists_term_ignore_case,count_ner_term,count_ner_term_ignore_case,count_for_org_claim
225,Antimicrobial agents are less effective due to...,Antimicrobial substances are less effective du...,0,0,0,0,0
226,Antimicrobial agents are less effective due to...,"Because of the pressure of antimicrobial use, ...",0,0,0,0,0
227,Antimicrobial agents are less effective due to...,Antimicrobial agents are less effective due to...,0,0,0,0,0
228,Antimicrobial agents are less effective due to...,Antimicrobial agents are less effective due to...,0,0,0,0,0
229,Antimicrobial agents are less effective due to...,Antimicrobial agents are less effective due to...,0,0,0,0,0
...,...,...,...,...,...,...,...
3366,The risk of male prisoners harming themselves ...,The risk of male prisoners harming themselves ...,0,0,0,0,0
3367,The risk of male prisoners harming themselves ...,The risk of male prisoners harming themselves ...,0,0,0,0,0
3368,The risk of male prisoners harming themselves ...,The risk of male prisoners posing themselves i...,0,0,0,0,0
3369,The risk of male prisoners harming themselves ...,The risk of male prisoners harming themselves ...,0,0,0,0,0


In [32]:
df_all_claim_selected_model_term_stat.groupby('org_claim')['count_exists_term', 
                                                                'count_exists_term_ignore_case', 
                                                                'count_ner_term', 
                                                                'count_ner_term_ignore_case', 
                                                                'count_for_org_claim'].aggregate(['mean', 'min', 'max']).aggregate(np.mean)

/home/qudratealahyratu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


count_exists_term              mean    1.794542
                               min     1.406069
                               max     2.112717
count_exists_term_ignore_case  mean    1.915468
                               min     1.575145
                               max     2.180636
count_ner_term                 mean    1.868523
                               min     1.471098
                               max     2.173410
count_ner_term_ignore_case     mean    1.997562
                               min     1.669075
                               max     2.241329
count_for_org_claim            mean    2.498555
                               min     2.498555
                               max     2.498555
dtype: float64

## Load from iterative models

In [365]:
#cur_model_outcome_dir = './results/paraphrased/paws/cumulative_threshold_07/'
cur_model_outcome_dir = '../../dfs_generated/paraphrased/paws/cumulative/'
cur_models_all_files =  glob.glob(os.path.join(cur_model_outcome_dir, '*.pkl'))

In [367]:
list_cur_itr_model_dfs = []
for cur_model_cur_file_name in cur_models_all_files:
    tmp_df = pd.read_pickle(cur_model_cur_file_name)
    tmp_df['file_name'] = os.path.basename(cur_model_cur_file_name)
    list_cur_itr_model_dfs.append(tmp_df)

In [368]:
df_cur_itr_model_all_res = pd.concat(list_cur_itr_model_dfs, axis=0, ignore_index=True)

In [369]:
df_cur_itr_model_all_res['file_name'].unique()

array(['paws_base_no_fine_tune_ParaphraseTargetDirection.org_support_to_gen_refute_19_FT_concat_prev.pkl',
       'paws_base_no_fine_tune_ParaphraseTargetDirection.org_support_to_gen_refute_14_FT_concat_prev.pkl',
       'paws_base_no_fine_tune_ParaphraseTargetDirection.org_support_to_gen_refute_17_FT_concat_prev.pkl',
       'paws_base_no_fine_tune_ParaphraseTargetDirection.org_support_to_gen_refute_11_FT_concat_prev.pkl',
       'paws_base_no_fine_tune_ParaphraseTargetDirection.org_support_to_gen_refute_1_FT_concat_prev.pkl',
       'paws_base_no_fine_tune_ParaphraseTargetDirection.org_support_to_gen_refute_21_FT_concat_prev.pkl',
       'paws_base_no_fine_tune_ParaphraseTargetDirection.org_support_to_gen_refute_18_FT_concat_prev.pkl',
       'paws_base_no_fine_tune_ParaphraseTargetDirection.org_support_to_gen_refute_10_FT_concat_prev.pkl',
       'paws_base_no_fine_tune_ParaphraseTargetDirection.org_support_to_gen_refute_24_FT_concat_prev.pkl',
       'paws_base_no_fine_tune_Paraphr

In [372]:
df_cur_itr_model_all_res['paraphrase_model'] = df_cur_itr_model_all_res['file_name'].progress_apply(lambda x: pd.Series(re.split('[_ .]',x)[0]))
df_cur_itr_model_all_res['fine_tune_direction'] = df_cur_itr_model_all_res['file_name'].progress_apply(lambda x: pd.Series('_'.join(re.split('[_ .]',x)[6:11])))
df_cur_itr_model_all_res['no_of_df_used_for_fine_tuned'] = df_cur_itr_model_all_res['file_name'].progress_apply(lambda x: pd.Series(re.split('[_ .]',x)[11]))

In [374]:
df_cur_itr_model_all_res.groupby(['org_claim', 'fine_tune_direction','no_of_df_used_for_fine_tuned'])

In [33]:
import neuspell
from neuspell import available_checkers, BertChecker

data folder is set to `/home/qudratealahyratu/research/nlp/fact_checking/existing_work/neuspell/neuspell/../data` script


In [44]:
checker = BertChecker(device="cuda")
checker.from_huggingface(bert_pretrained_name_or_path="emilyalsentzer/Bio_ClinicalBERT")

TypeError: from_huggingface() missing 1 required positional argument: 'vocab'

In [47]:
'aPKCz causes tumour suppression by affecting glutamine metabolism.'.lower().

'apkcz causes tumour suppression by affecting glutamine metabolism.'

In [42]:
import contextualSpellCheck